In [1]:
import pandas as pd
import json

def read_eval(path):
    return pd.read_json(path, orient="index")

In [2]:
def get_number_compile(df):
    count = 0
    all_count = 0
    for index, row in df.iterrows():
        for fix in row.fixes:
            all_count += 1
            if fix["comp_pass"] and row.buggy_line != row.fixed_line:
                count += 1
    return "%d / %d (%.2f%%)" % (count, all_count, count/all_count)

In [3]:
def get_number_fixed(df):
    count = 0
    for index, row in df.iterrows():
        for i, fix in enumerate(row.fixes):
            if fix["test_pass"] and row.buggy_line != row.fixed_line:
                count += 1
                break
    return "%d / %d (%.2f%%)" % (count, len(df), count/len(df))

In [4]:
def get_table_entry(beam_width, critic, round_, tbb, tbs, tdb, tds, cb, tb, cd, td):
    return {"Hyper Parameters": {
                "" :   { "Beam Width" : beam_width }
            },
            "Training": {
                "Bears" : {"BugLab" : tbb, "SelfAPR": tbs},
                "Defects4J" : {"BugLab" : tdb, "SelfAPR": tds},
                "Critic" : { "" : critic },
                "Round" : { "" : round_ }
            },
            "Testing":{
                "Bears Compile" : { "" : cb },
                "Bears Test Pass" : { "" : tb },
                "Defects4J Compile" : { "" : cd},
                "Defects4J Tests Pass" : { "" : td}
            }
       }

In [5]:
entries = []

path = "/home/andre/Repos/mscthesis/experiments/defects4j_buglab/round0/fixer/eval_defects4j_bw_10.json"
cb = "N/A"
tb = "N/A"
cd = get_number_compile(read_eval(path))
td = get_number_fixed(read_eval(path))
entries += [get_table_entry(10, "nocritic", "round1", "", "", "x", "", cb, tb, cd, td)]

path = "/home/andre/Repos/mscthesis/experiments/defects4j_buglab/round1-nocritic/part2/fixer/eval_defects4j_bw_10.json"
cb = "N/A"
tb = "N/A"
cd = get_number_compile(read_eval(path))
td = get_number_fixed(read_eval(path))
entries += [get_table_entry(10, "nocritic", "round1", "", "", "x", "", cb, tb, cd, td)]

In [6]:
table = pd.json_normalize(data=entries)
table.columns = table.columns.str.split(".").map(tuple)
table

Hyper Parameters Training                                              \
                      Bears         Defects4J            Critic   Round   
        Beam Width   BugLab SelfAPR    BugLab SelfAPR                     
0               10                          x          nocritic  round1   
1               10                          x          nocritic  round1   

        Testing                                                           
  Bears Compile Bears Test Pass   Defects4J Compile Defects4J Tests Pass  
                                                                          
0           N/A             N/A  501 / 4290 (0.12%)     16 / 429 (0.04%)  
1           N/A             N/A  450 / 4290 (0.10%)     14 / 429 (0.03%)